In [5]:
import pandas as pd

import statsmodels.api as sm
import pandas as pd
import datetime
from pathlib import Path

pd.set_option("display.max_columns", None)
# !pip install statsmodels


# %config InlineBackend.figure_format = "svg"
# %config InlineBackend.print_figure_kwargs = {"dpi" : 300}
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


# from cadFace.vis import percentiles_plot
import sci_palettes

try:
    sci_palettes.register_cmap()
except:
    pass
import scienceplots
from pathlib import Path
import pandas as pd
from ppp_aging.ppp_model import *
from ppp_aging.model import generate_states_cols
import json

plt.style.use(["nature", "no-latex"])
sns.set_context("paper", font_scale=1.5)
sns.set_palette("nejm")

In [22]:
from cuml import LogisticRegression, Lasso, Ridge, ElasticNet
from cuml.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer


def fit_best_model(train_df, test_df, X_var, y_var, method_list=None, cv=10, verbose=1):
    models_params = {
        "Logistic": {
            "model": LogisticRegression(
                solver="qn", random_state=42, class_weight="balanced"
            ),
            "param_grid": {
                "C": np.logspace(-4, 4, 10),  # C参数的范围，使用对数间隔
                "penalty": ["l1", "l2"],  # 正则化类型
            },
        },
        "Lasso": {
            "model": Lasso(),
            "param_grid": {
                "alpha": np.logspace(-4, 4, 10),
            },
        },
        "ElasticNet": {
            "model": ElasticNet(),
            "param_grid": {
                "alpha": np.logspace(-4, 4, 10),
                "l1_ratio": np.linspace(0, 1, 10),
            },
        },
        # "RandomForest": {
        #     "model": RandomForestClassifier(),
        #     "param_grid": {"n_estimators": range(10, 101, 10)},
        # },
    }
    if method_list is not None:
        models_params = {k: v for k, v in models_params.items() if k in method_list}

    train_df = train_df[[y_var] + X_var].copy().dropna()
    test_df = test_df[[y_var] + X_var].copy().dropna()
    train_df[y_var] = train_df[y_var].astype(int)
    test_df[y_var] = test_df[y_var].astype(int)

    train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

    X_train = train_df[X_var]
    y_train = train_df[y_var]
    X_val = val_df[X_var]
    y_val = val_df[y_var]

    X_test = test_df[X_var]
    y_test = test_df[y_var]
    print(
        f"train shape: {X_train.shape}, val shape is {X_val.shape}, test shape is {X_test.shape}"
    )
    best_models = []

    for model_name, mp in models_params.items():
        # if model_name == "RandomForest":
        #     best_model = RandomForestClassifier(verbose=verbose)
        #     best_model.fit(X_train.values, y_train.values)
        #     auc = roc_auc_score(y_val, best_model.predict(X_val.values))
        #     bset_params = None  # no params for RandomForest

        # else:
        if model_name == "Logistic":
            scorer = make_scorer(roc_auc_score, needs_proba=True)
        else:
            scorer = make_scorer(roc_auc_score)

        grid_search = GridSearchCV(
            mp["model"], mp["param_grid"], scoring=scorer, cv=cv, verbose=verbose
        )
        grid_search.fit(X_train.values, y_train.values)

        best_model = grid_search.best_estimator_
        bset_params = grid_search.best_params_

        if model_name == "Logistic":
            auc = roc_auc_score(y_val, best_model.predict_proba(X_val.values)[:, 1])
        else:
            auc = roc_auc_score(y_val, best_model.predict(X_val.values))
        print(f"model: {model_name}\tBest parameters: {bset_params}, with auc: {auc}")
        best_models.append((model_name, best_model, grid_search, auc))

    ## select the currently best
    # print(best_models)

    # 还原原始的train_df
    train_df = pd.concat([train_df, val_df], axis=0)
    X_train = train_df[X_var]
    y_train = train_df[y_var]

    best_mdoels = list(sorted(best_models, key=lambda x: x[-1], reverse=True))
    best_model_name, best_model, *_ = best_mdoels[0]

    if best_model_name == "Logistic":
        train_pred = best_model.predict_proba(X_train.values)[:, 1]

        test_pred = best_model.predict_proba(X_test.values)[:, 1]
    else:
        train_pred = best_model.predict(X_train.values)
        val_pred = best_model.predict(X_val.values)
        test_pred = best_model.predict(X_test.values)

    train_df[f"{y_var}_pred"] = train_pred

    test_df[f"{y_var}_pred"] = test_pred

    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)

    train_metrics = {
        "train_auc": train_auc,
    }
    test_metrics = {
        "test_auc": test_auc,
    }
    return best_model, train_metrics, test_metrics, train_df, test_df, best_mdoels

## 读入数据

In [23]:
risk_factors = [
    "age",
    "sex",
    "ldl_a",
    "hdl_a",
    "tc_a",
    "tg_a",
    "apob",
    "sbp_a",
    "dbp_a",
]

E = "incident_cad"
T = "future_cad_time_months"

In [11]:
train_imputed = pd.read_pickle("1_train_imputed.pkl")
test_imputed = pd.read_pickle("1_test_imputed.pkl").dropna(
    subset=["PRS"] + risk_factors
)

X_combination = json.load(open("1_X_combination_dict.json"))

In [14]:
print(f"X_combination have {len(X_combination)} keys")
print(
    f"train_imputed shape is {train_imputed.shape}, test_imputed shape is {test_imputed.shape}"
)
print(
    f"train have {E}: {train_imputed[E].value_counts()}; test have {E}: {test_imputed[E].value_counts()}"
)

X_combination have 22 keys
train_imputed shape is (40806, 2966), test_imputed shape is (8298, 2966)
train have incident_cad: 0.0    38544
1.0     2262
Name: incident_cad, dtype: int64; test have incident_cad: 0.0    7859
1.0     439
Name: incident_cad, dtype: int64


In [19]:
from collections import OrderedDict

X_combination = OrderedDict(sorted(X_combination.items(), key=lambda x: len(x[1])))
X_combination

OrderedDict([('age_sex', ['age', 'sex']),
             ('PRS_age_sex', ['PRS', 'age', 'sex']),
             ('risk_factors',
              ['age',
               'sex',
               'ldl_a',
               'hdl_a',
               'tc_a',
               'tg_a',
               'apob',
               'sbp_a',
               'dbp_a']),
             ('PRS_risk_factors',
              ['PRS',
               'age',
               'sex',
               'ldl_a',
               'hdl_a',
               'tc_a',
               'tg_a',
               'apob',
               'sbp_a',
               'dbp_a']),
             ('single_surivival_sig_multivar_res_sig',
              ['NTproBNP',
               'MMP12',
               'VWC2',
               'SUSD2',
               'SIGLEC8',
               'EPS8L2',
               'CHGA',
               'ACTA2',
               'REN',
               'IL1RN',
               'RNASE4',
               'HSPB6',
               'CD99L2']),
             ('single_su

## fit best model

In [20]:
from collections import defaultdict
import pickle
from pathlib import Path

Regression_model_result_dict = defaultdict(dict)
output = "./1_regression_models"
Path(output).mkdir(exist_ok=True)

In [24]:
# %%time
methods = ["Lasso", "ElasticNet", "Logistic"]
cv = 10
for k, combination in X_combination.items():
    if k in Regression_model_result_dict.keys():
        print(f"{k} already in result dict")
        continue
    (
        model,
        train_metrics,
        test_metrics,
        train_imputed_data,
        test_imputed_data,
        best_models,
    ) = fit_best_model(
        train_df=train_imputed,
        test_df=test_imputed,
        X_var=X_combination[k],
        y_var=E,
        method_list=methods,
        cv=cv,
    )
    print(
        f"{k} train auc: {train_metrics['train_auc']}, test auc: {test_metrics['test_auc']}"
    )
    all_obj = {
        "model": model,
        "train_metrics": train_metrics,
        "test_metrics": test_metrics,
        "train_data": train_imputed_data,
        "test_data": test_imputed_data,
        "best_models": best_models,
    }
    Regression_model_result_dict[k] = all_obj
    pickle.dump(Regression_model_result_dict, open(f"{output}/{k}.pkl", "wb"))

[I] [15:43:56.296896] Unused keyword parameter: random_state during cuML estimator initialization
train shape: (36722, 2), val shape is (4081, 2), test shape is (8298, 2)
Fitting 10 folds for each of 20 candidates, totalling 200 fits


/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


model: Logistic	Best parameters: {'C': 0.046415888336127774, 'penalty': 'l2'}, with auc: 0.7381355232750083
Fitting 10 folds for each of 10 candidates, totalling 100 fits
model: Lasso	Best parameters: {'alpha': 0.000774263682681127}, with auc: 0.7393122116834564
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
model: ElasticNet	Best parameters: {'alpha': 0.000774263682681127, 'l1_ratio': 1.0}, with auc: 0.7393122116834564
age_sex train auc: 0.7182158786384111, test auc: 0.7168387244315457
[I] [15:45:08.153401] Unused keyword parameter: random_state during cuML estimator initialization
train shape: (36722, 3), val shape is (4081, 3), test shape is (8298, 3)
Fitting 10 folds for each of 20 candidates, totalling 200 fits


/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


model: Logistic	Best parameters: {'C': 21.54434690031882, 'penalty': 'l2'}, with auc: 0.7707141773665378
Fitting 10 folds for each of 10 candidates, totalling 100 fits
model: Lasso	Best parameters: {'alpha': 0.000774263682681127}, with auc: 0.7711513923960276
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
model: ElasticNet	Best parameters: {'alpha': 0.005994842503189409, 'l1_ratio': 0.2222222222222222}, with auc: 0.770850249391022
PRS_age_sex train auc: 0.7545376310014774, test auc: 0.7436109841422034
[I] [15:46:33.908085] Unused keyword parameter: random_state during cuML estimator initialization
train shape: (29878, 9), val shape is (3320, 9), test shape is (8298, 9)
Fitting 10 folds for each of 20 candidates, totalling 200 fits


...

[W] [12:51:37.108450] L-BFGS: max iterations reached
[W] [12:51:37.108639] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [12:51:40.826753] L-BFGS: max iterations reached
[W] [12:51:40.827131] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [12:52:32.000394] L-BFGS: max iterations reached
[W] [12:52:32.000679] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [12:52:38.986890] L-BFGS: max iterations reached
[W] [12:52:38.987187] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the inpu

/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


[W] [13:39:11.534985] L-BFGS: max iterations reached
[W] [13:39:11.535196] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [13:39:24.969044] L-BFGS: max iterations reached
[W] [13:39:24.969228] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [13:39:56.763563] L-BFGS: max iterations reached
[W] [13:39:56.763906] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [13:40:00.717383] L-BFGS: max iterations reached
[W] [13:40:00.717787] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the inpu

/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


[W] [14:18:49.177527] QWL-QN stopped, because the line search failed to advance (step delta = 0.000000)
[W] [14:18:53.447639] QWL-QN stopped, because the line search failed to advance (step delta = 0.000000)
[W] [14:22:40.657644] QWL-QN: max iterations reached
[W] [14:22:40.658229] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [14:22:53.236853] QWL-QN: max iterations reached
[W] [14:22:53.237776] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [14:23:05.348252] QWL-QN: max iterations reached
[W] [14:23:05.348780] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [14:23:19.087879] QWL-QN: max it